<a href="https://colab.research.google.com/github/poojaneuusa/NLP-Projects/blob/main/Question_Answer_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer, util
import os
import json
import gzip

In [3]:
util.http_get('https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/simplewiki-2020-11-01.jsonl.gz', 'simplewiki-2020-11-01.jsonl.gz')

  0%|          | 0.00/50.2M [00:00<?, ?B/s]

In [4]:
model = SentenceTransformer('nq-distilbert-base-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
passages = []
with gzip.open('simplewiki-2020-11-01.jsonl.gz', 'rt', encoding='utf8') as file:
  for line in file:
    data = json.loads(line.strip())
    # print(data)
    for paragraph in data['paragraphs']:
      passages.append([data['title'], paragraph])
    # break

In [6]:
len(passages)

509663

In [7]:
from random import shuffle
shuffle(passages)

In [8]:
passages = passages[0:100_000]

In [9]:
corpus_embeddings = model.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

In [10]:
def get_answer(query):

  question_embedding = model.encode(query, convert_to_tensor=True)
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=3)[0]

  print("Results:")
  for hit in hits:
    print(passages[hit['corpus_id']])

In [13]:
get_answer("what is the ML?")

Results:
['OCaml', 'Objective Caml (OCaml) is a programming language which is a dialect of ML (programming language). It extends the Caml language so object-oriented programming can be used.']
['Standard ML', 'Standard ML is a functional programming language which is a dialect of ML (programming language). It is sometimes used for writing compilers and in theorem provers.']
['Litre', 'A litre (international spelling) or liter (American spelling) is one of the metric units of volume. It is not an SI unit.']


In [12]:
get_answer("when did the first world war end?")

Results:
['World War I', 'World War I (WWI or WW1), also called the First World War, began on July 28, 1914 and lasted until November 11, 1918. The war was a global war that lasted exactly . Most of the fighting was in Europe, but soldiers from many other countries took part, and it changed the colonial empires of the European powers. Before World War II began in 1939, World War I was called the Great War or the World War. 135 countries took part in World War I, and nearly 10 million people died while fighting.']
['History of the world', 'On November 11, 1918, Germany signed the armistice, meaning "the laying down of arms", to end the war. After the war ended, the Treaty of Versailles was written and Germany was made to sign it. They had to pay $33 million in reparations (payment for damage). The influenza pandemic of 1918 spread around the world, killing millions.']
['World War I', "Before the war, European countries had formed alliances with each other to protect themselves. However,